In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file

scaler = StandardScaler()

scaled_data = scaler.fit_transform(df_market_data.select_dtypes(include=['float64', 'int64']))
scaled_data.shape


In [ ]:
# Create a DataFrame with the scaled data
#df_scaled = pd.DataFrame(scaled_data, columns=df_market_data.select_dtypes(include=['float64', 'int64']).columns, index=df_market_data.index)
df_scaled = pd.DataFrame(scaled_data,columns = df_market_data.columns )

# Copy the crypto names from the original DataFrame
df_scaled['coin_id'] = list(df_market_data.index)

# Set the coin_id column as index
df_scaled = df_scaled.set_index('coin_id')

# Display the scaled DataFrame
df_scaled.head()

---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))  


In [ ]:
# Create an empty list to store the inertia values
inertia_values = []


# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
import os
os.environ["OMP_NUM_THREADS"] = "1"

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0,n_init='auto')
    kmeans.fit(df_scaled)
    inertia_values.append(kmeans.inertia_)


In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k": k_values,
    "inertia": inertia_values
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow1 = pd.DataFrame(elbow_data)


In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_elbow1.hvplot.line(
    x="k",
    y="inertia",
    title= "Elbow Curve",
    xticks=k
)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:**  4

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [ ]:
# Initialize the K-Means model using the best value for k
best_k = 4
kmeans = KMeans(n_clusters=best_k, random_state=0, n_init=10)


In [ ]:
# Fit the K-Means model using the scaled DataFrame
kmeans.fit(df_scaled)


In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
cluster_labels = kmeans.predict(df_scaled)


# Print the resulting array of cluster values.
cluster_labels 

In [ ]:
# Create a copy of the scaled DataFrame
df_scaled_copy1 = df_scaled.copy()

In [ ]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_scaled_copy1["cluster"] = cluster_labels


# Display the copy of the scaled DataFrame
df_scaled_copy1

In [ ]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df_scaled_copy1.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='cluster',
    hover_cols = 'coin_id'
)

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)


In [ ]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
pca_data = pca.fit_transform(df_scaled)


# View the scaled PCA data
pca_data

In [ ]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_


In [ ]:
sum(pca.explained_variance_ratio_)

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:**  0.8950316570309841

In [ ]:
# Create a new DataFrame with the PCA data.
df_pca = pd.DataFrame(pca_data,    columns = ["PC1", "PC2", "PC3"])
# Copy the crypto names from the original scaled DataFrame
df_pca['coin_id'] =list(df_scaled.index)

# Set the coin_id column as index
df_pca = df_pca.set_index('coin_id')

# Display the scaled PCA DataFrame
df_pca

---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12)) 


In [ ]:
# Create an empty list to store the inertia values
inertia_values = []


# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0) 
    kmeans.fit(df_pca)
    inertia_values.append(kmeans.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k": k_values,
    "inertia": inertia_values
}
# Create a DataFrame with the data to plot the Elbow curve
df_elbow2 = pd.DataFrame(elbow_data)


In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_elbow2.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:**   No

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [ ]:
#df_pca

In [ ]:
# Initialize the K-Means model using the best value for k
best_k = 4

In [ ]:
# Fit the K-Means model using the PCA data
kmeans = KMeans(n_clusters=best_k, random_state=0, n_init=10)
kmeans.fit(df_pca)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
cluster_labels = kmeans.predict(df_pca)


# Print the resulting array of cluster values.
cluster_labels

In [ ]:
# Create a copy of the scaled PCA DataFrame
df_pca_copy2 = df_pca.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
df_pca_copy2["cluster"] = cluster_labels


# Display the copy of the scaled PCA DataFrame
df_pca_copy2

In [ ]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.

scatter_plot = df_pca_copy2.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="cluster",
    hover_cols='coin_id',
    title="Cryptocurrency Clustering"
)

scatter_plot

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
# YOUR CODE HERE!
df_elbow1.hvplot.line(
    x="k",
    y="inertia",
    title= "scaled Elbow Curve",
    xticks=k
) +  df_elbow2.hvplot.line(
    x="k", 
    y="inertia", 
    title="scaled pca Elbow Curve", 
    xticks=k
)


In [ ]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!

df_scaled_copy1.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='cluster',
    hover_cols = 'coin_id',
    title=" scaled Cryptocurrency Clustering"
) +  df_pca_copy2.hvplot.scatter(
    x="PC1",
    y="PC2",
    by='cluster',
    hover_cols='coin_id',
    title="scaled pca Cryptocurrency Clustering"
)


#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:**  The boundaries of the clustering results are clearer, and the points within the clusters are more concentrated.